In [1]:
from fla.layers.kda import KimiDeltaAttention
import torch

In [2]:
batch, seq, dim = 2, 128, 512
num_heads = 8

assert dim % num_heads == 0

headdim = dim // num_heads

In [3]:
attn = KimiDeltaAttention(hidden_size=dim, num_heads=num_heads)

In [4]:
x = torch.randn(batch, seq, dim)

In [5]:
attn = attn.to("cuda")
x = x.to("cuda")

In [6]:
out = attn(x)

In [ ]:
l = out[0].mean()
l.backward()